# Configuration

A MetaDrive instance accepts a dict as the environmental config. For example, you can build a MetaDrive instance with 200 generated maps via 
```python
from metadrive import MetaDriveEnv
config = dict(num_scenarios=200, start_seed=0)
env = MetaDriveEnv(config)
```

In this page, we describe the details of the config system and configurable options for all environments.

## Config system
This section discusses how to configure the an environment in MetaDrive and some features of the config system.

### Overwriting
Every environment has a default config, which records the parameters required to launch the environment. It is content is actually a nested dictionary whose keys and values represent the parameter names and corresponding values. 
This default config dict can be accessed via the class method:
```python
default_config = MetaDriveEnv.default_config()
```

When creating environments, the external config `config` will overwritten default values of certain fields in the `default_config`. The following code exemplifies this. 

In [30]:
from metadrive import MetaDriveEnv
default_config = MetaDriveEnv.default_config()
env = MetaDriveEnv(dict(num_scenarios=100, log_level=50))
env_config = env.config
print("default_config['num_scenarios']:", default_config["num_scenarios"])
print("env_config['num_scenarios']:", env_config["num_scenarios"])

default_config['num_scenarios']: 1
env_config['num_scenarios']: 100


### Sanity Check
There is a check mechanism which prohibit users to set the value for a key that doesn't exist in the `default_config`. This is helpful to make sure that users type the correct parameter name and successfully config the target parameter. 

In [20]:
try:
    env = MetaDriveEnv(dict(non_exist_key=False))
except KeyError as error:
    print(str(error)[:62] + " ...")

"'{'non_exist_key'}' does not exist in existing config. Please ...


The check mechanism will further ensure if the type of the parameter is correct. For example, the `num_scenarios` should be an `int` type, and thus a `list` type parameter will raise an error.   

In [28]:
try:
    env = MetaDriveEnv(dict(num_scenarios=[0, 1]))
except AssertionError as error:
    print(str(error)[:62] + " ...")

TypeError: num_scenarios:[0, 1] ...


### Basic Config Sharing
The default configs are different across all environments, but may share some identical fields. Take the `MetaDriveEnv` and `ScenarioEnv` as example.

In [9]:
from metadrive.envs import MetaDriveEnv, ScenarioEnv
metadrive_config = set(MetaDriveEnv.default_config().keys())
scenario_config = set(ScenarioEnv.default_config().keys())
print("Number of parameters of MetaDriveEnv: {}".format(len(metadrive_config)))
print("Number of parameters of ScenarioEnv: {}\n".format(len(scenario_config)))

try:
    assert metadrive_config == scenario_config
except AssertionError as error:
    print("The config between MetaDriveEnv and ScenarioEnv is different.\n")
    
identical_parameters = scenario_config.intersection(metadrive_config)
print("Number of identical parameters: \
      {}".format(len(identical_parameters)))
print("Number of unique parameters in MetaDriveEnv: \
      {}".format(len(metadrive_config-identical_parameters)))
print("Number of unique parameters in ScenarioEnv: \
      {}".format(len(scenario_config-identical_parameters)))

Number of parameters of MetaDriveEnv: 109
Number of parameters of ScenarioEnv: 121

The config between MetaDriveEnv and ScenarioEnv is different.

Number of identical parameters:       91
Number of unique parameters in MetaDriveEnv:       18
Number of unique parameters in ScenarioEnv:       30


It is worth mentioning the parameter sharing mechanism, which is helpful when we create a new environment, so we don't need to copy some common configs to the `default_config` to the new environments again and again. Let's first check out how the `default_config()` function is implemented.

In [10]:
from metadrive.utils import print_source
print_source(ScenarioEnv.default_config)

@classmethod
def default_config(cls):
    config = super(ScenarioEnv, cls).default_config()
    config.update(SCENARIO_ENV_CONFIG)
    return config



It is quite simple and is implemented by overwriting the `super(ScenarioEnv, cls).default_config()` with `Scenario_ENV_CONFIG`. If we check the contents of the two config dict, we will find that the `BaseEnv.default_config() = super(ScenarioEnv, cls).default_config()` is the subset of `ScenarioEnv.default_config()` and provides the `ScenarioEnv` with the basic configs.

In [22]:
from metadrive.envs.base_env import BaseEnv
set(BaseEnv.default_config()).issubset(set(ScenarioEnv.default_config()))

True

It is the same for the MetaDriveEnv as well, whose default config is:

In [23]:
print_source(MetaDriveEnv.default_config)

@classmethod
def default_config(cls) -> Config:
    config = super(MetaDriveEnv, cls).default_config()
    config.update(METADRIVE_DEFAULT_CONFIG)
    config.register_type("map", str, int)
    config["map_config"].register_type("config", None)
    return config



As there is an overwriting function is called, it is ok to overwrite the values of parameters in `BaseEnv.default_config()` when making the `default_config` for a inherited environment. The following code shows that the config `show_sidewalk` is *True* in `BaseEnv` but is overwritten to *False* in `ScenarioEnv` because of the `SCENARIO_ENV_CONFIG`.

In [25]:
from metadrive.envs.scenario_env import SCENARIO_ENV_CONFIG

assert BaseEnv.default_config()["show_sidewalk"]
assert not ScenarioEnv.default_config()["show_sidewalk"]
assert not SCENARIO_ENV_CONFIG["show_sidewalk"]

## Basic Configs

As all environments are subclass of `BaseEnv` and share the parameters of `BaseEnv`, we first discuss the parameters in `BaseEnv.default_config()`. 
The available items with annotations are listed as follows.
You can check this in the source code as well.

In [1]:
import metadrive.envs.base_env as base_env
from metadrive.utils import print_source, CONFIG
module_source = print_source(base_env, ["BASE_DEFAULT_CONFIG", ")\n\n"], colorscheme=CONFIG)

BASE_DEFAULT_CONFIG = dict(

    # ===== agent =====
    random_agent_model=False,  # randomize the car model for the agent, randomly choosing from 4 types of cars
    target_vehicle_configs={DEFAULT_AGENT: dict(use_special_color=True, spawn_lane_index=None)},

    # ===== multi-agent =====
    num_agents=1,  # Note that this can be set to >1 in MARL envs, or set to -1 for as many vehicles as possible.
    is_multi_agent=False,
    allow_respawn=False,
    delay_done=0,  # How many steps for the agent to stay static at the death place after done.

    # ===== Action/Control =====
    manual_control=False,  # if set to True, agent policy will be Manual Control policy
    controller="keyboard",  # "steering_wheel" or "keyboard" or "xbos"
    agent_policy=EnvInputPolicy,
    discrete_action=False,
    discrete_steering_dim=5,
    discrete_throttle_dim=5,
    use_multi_discrete=False,  # If True, use MultiDiscrete action space. Otherwise, use Discrete.
    action_check=False,  # check if t



### 

MetaDrive can generate or load unlimited number of driving scenarios by using procedural generation and real-world driving data, respectively.

We can specify the range of the scenarios used in different tasks.
For example, you can use different set of generated scenarios to train and test the trained agents.
To achieve that, you only need to specify the range of random seeds used to generated those scenarios.
Concretely, MetaDrive will use the seeds in range :code:`[start_seed, start_seed + environment_num)`.
Therefore, you only need to specify these two values in the config:

    - :code:`start_seed` (int = 0): random seed of the first map
    - :code:`environment_num` (int = 1): number of the driving scenarios





Map Config
#############

MetaDrive provides detailed configuration on the generated maps. Generally speaking, we allow two forms of map generation if using Procedural Generation (PG) algorithm while not loading map from dataset:

1. :code:`config["map_config"]["type"] = "block_num"`: the user specifies the number of blocks in each map so that the PG algorithm will automatically build maps containing that number of blocks while randomizing all parameters including the type of blocks.
2. :code:`config["map_config"]["type"] = "block_sequence"`: the user specify the sequence of block types and PG algorithm will build maps strictly following that order while randomizing the parameters in each block.

We describe all optional map config as follows:

    - :code:`random_lane_width` (bool = False): whether to randomize the width of lane in each map (all lanes in the same map share the same lane width)
    - :code:`random_lane_num` (bool = False): whether to randomize the number of lane in a road in each map (all road in the same map share the same number of lanes)
    - :code:`map_config` (dict): A nested dict describing the generation of map.
        - :code:`type` (str = "block_num"): A string in ["block_num", "block_sequence"] denoting which form of map generation should the PG algorithm use.
        - :code:`config`: XXX
        - :code:`lane_width` (float = 3.5): the width of each lane. This will be overwritten if :code:`random_lane_width = True`.
        - :code:`lane_num` (int = 3): number of lanes in each road. This will be overwritten if :code:`random_lane_num = True`.
        - :code:`exit_length` (float = 50): more than one exit whose length is *exit_length* are contained in some blocks like roundabout


We also provide a shortcut to specify the map:

    -   :code:`map` (int or string): User can set a *string* or *int* as the key to generate map in an easy way. For example, :code:`config["map"] = 3` means generating a map containing 3 blocks, while :code:`config["map"] = "SCrRX"` means the first block is Straight, and the following blocks are Circular, InRamp, OutRamp and Intersection. The character here are the unique ID of different types of blocks as shown in the next table. Therefore using a *string* can determine the block type sequence.
        We provide the following block types:

            +---------------+-----------+
            | Block Type    |    ID     |
            +===============+===========+
            | Straight      |     S     |
            +---------------+-----------+
            | Circular      |     C     |
            +---------------+-----------+
            | InRamp        |     r     |
            +---------------+-----------+
            | OutRamp       |     R     |
            +---------------+-----------+
            | Roundabout    |     O     |
            +---------------+-----------+
            | Intersection  |     X     |
            +---------------+-----------+
            | TIntersection |     T     |
            +---------------+-----------+
            | Merge         |     y     |
            +---------------+-----------+
            | Split         |     Y     |
            +---------------+-----------+
            | Tollgate      |     $     |
            +---------------+-----------+
            | Parking-lot   |     P     |
            +---------------+-----------+
            | Fork          |    WIP    |
            +---------------+-----------+






Action Config
##############

    - :code:`manual_control` (bool = False): whether to control ego vehicle by user in the interface (require :code:`use_render = True`)
    - :code:`controller` (str = "keyboard"): select in ["keyboard", "joystick"], the controller for user to control the ego vehicle
    - :code:`discrete_action` (bool = False): whether to discretize the action space
    - :code:`use_multi_discrete` (bool = False): whether to MultiDiscrete action space, if `discrete_action=True`
    - :code:`discrete_steering/throttle_dim` (int = 5, 5): how many dimensions used to discrete the action space
    - :code:`decision_repeat` (int): how many times for the simulation engine to repeat the applied action to the vehicles. The minimal simulation interval :code:`physics_world_step_size` is 0.02 s. Therefore each RL step will last :code:`decision_repeat * 0.02 s` in the simulation world.


Agent Config
#############

    - :code:`random_agent_model` (bool = False): whether to randomize the dynamic model of ego vehicle
    - :code:`IDM_agent` (bool = False): whether to control ego vehicle by IDM policy





Visualization & Rendering Config
##################################

The config in this part specifies the setting related to visualization. The :code:`use_render` is the most useful one.

    - :code:`use_render` (bool = False): whether to pop a window on your screen or not. This is irrelevant to the vision-based observation.
    - :code:`disable_model_compression` (bool = True): Model compression reduces the memory consumption when using Panda3D window to visualize. Disabling model compression greatly improves the launch speed but might cause breakdown in low-memory machine.
    - :code:`cull_scene` (bool = True): When you want to access the image of camera, it should be set to True.
    - :code:`use_chase_camera_follow_lane` (bool = False): whether to force the third-person view camera following the heading of current lane
    - :code:`camera_dist/height` (float = 6.0, 1.8): the initial distance and height of the third-person view camera
    - :code:`prefer_track_agent` (str = None): specify the name of the agent that you wish to track in the third-person view. This is useful in the visualization in multi-agent environments.
    - :code:`top_down_camera_initial_x/y/z` (int = 0, 0, 200): the initial position of the top-down view camera


Vehicle Config
################

We list the vehicle config here. Observation Space will be adjusted by these config automatically. For example, if you set :code:`config["vehicle_config"]["lidar"]["num_lasers"] = 720`, then the dimension of the Lidar observation will automatically set to 720.

    - :code:`vehicle_config` (dict):
        - :code:`lidar` (dict): the config is related to the :ref:`Lidar-like observation <observation:State Vector>`. This Lidar only scans nearby vehicles.
            - :code:`num_lasers` (int = 240): the number of lasers used in Lidar
            - :code:`distance` (float = 50.0): the perception field radius
            - :code:`num_others` (int = 0): if this is greater than 0, MetaDrive will retrieve the states of :code:`num_others`-nearest vehicles as additional information
            - :code:`gaussian_noise` (float = 0.0): if this is greater than 0, MetaDrive will add Gaussian noise with :code:`gaussian_noise` standard deviation to each entry of the Lidar cloud points
            - :code:`dropout_prob` (float = 0.0): in [0, 1]. If this is greater than 0, MetaDrive will randomly set :code:`dropout_prob` % of entries in the cloud points to zero
        - :code:`side_detector` (dict): This Lidar only scans the side of the road but not vehicles. The config dict has identical keys as :code:`lidar` except :code:`num_others`.
        - :code:`lane_line_detector` (dict): This Lidar only scans the side of current lane but neither vehicles or road boundary. The config dict has identical keys as :code:`lidar` except :code:`num_others`.
        - :code:`show_lidar` (bool = False): whether to show the end of each Lidar laser in the scene
        - :code:`vehicle_model` (str = "default"): which type of vehicle to use in ego vehicle (s, m, l, xl, default)
        - :code:`enable_reverse` (bool = False): If True and vehicle speed < 0, a brake action (e.g. acceleration = -1) will be parsed as reverse. This is used in the Multi-agent Parking Lot environment.
        - :code:`random_color` (bool = False): whether to randomize the color of ego vehicles. This is useful in multi-agent environments.
        - :code:`image_source` (str = "rgb_camera"): select in ["rgb_camera", "depth_camera"]. When using image observation, it decides where the image collected. See :ref:`use_native_rendering` for more information.
        - :code:`rgb_camera` (tuple = (84, 84): (camera resolution width (int), camera resolution height (int). We use (84, 84) as the default size so that the RGB observation is compatible to those CNN used in Atari. Please refer to :ref:`use_native_rendering` for more information about using image as observation.
        - :code:`spawn_lane_index` (tuple): which lane to spawn this vehicle. Default to one lane in the first block of the map
        - :code:`spawn_longitude/lateral` (float = 5.0, 0.0): The spawn point will be calculated by *spawn_longitude* and *spawn_lateral*
        - :code:`destination` (str = None): the destination road node name. This is used in real dataset replay map.
        - :code:`mini_map` (tuple): (camera resolution width(int), camera resolution height(int), camera height). The size of the bird-view image in the left upper corner of the interface.





Other Observation Config
##########################

The vehicle config decides many of the observational config.

    - :code:`image_observation` (bool = False): If you want to use vision-based observation, please set this to True. See :ref:`use_native_rendering` for more information.
    - :code:`norm_pixel` (bool = True): if True than squeeze the value between \[0, 255\] to \[0.0, 1.0\]

Traffic Config
##################################


Currently, MetaDrive provides two built-in traffic modes: Respawn mode and Trigger mode.


In Respawn mode, Traffic Manager assigns traffic vehicles to random spawn points on the map.
The vehicles immediately start driving toward their destinations after spawning.
When a traffic vehicle terminates, it will be re-positioned to an available spawn point.
Respawn traffic mode is designed to maintain traffic flow density.

On the contrary, the Trigger mode traffic flow is designed to maximize the interaction between target vehicles and traffic vehicles.
The vehicles stay still in the spawn points until the target agent enters the trigger zone in each block.
Take an Intersection block as an case, the traffic vehicles inside the intersection will be triggered and start moving only when the target vehicle trespasses into the intersection.

Here we provide many config to adjust the traffic flow. Note that you can even setup rule-based traffic flow by setting :code:`traffic_mode` > 0.


    - :code:`traffic_density` (float = 0.1): number of traffic vehicles per 10 meter per lane
    - :code:`traffic_mode` (str = "Trigger"): select in ["Trigger", "Respawn"]
    - :code:`random_traffic` (bool = False): If set to False, each driving scenario will have deterministic traffic flow. Otherwise the traffic generation will not be controlled by current seed and provide various traffic flow even in the same road network.


Multi-agent Config
##################


    - :code:`num_agents` (int = 1): the number of agent that are controllable by RL policies
    - :code:`is_multi_agent` (bool = False): set this to True if in multi-agent training (default to True in MA)
    - :code:`allow_respawn` (bool = False): whether allow (default to True in MA)
    - :code:`delay_done` (int = 0): how many environmental steps for the agent to stay static as an obstacle after it is terminated (default to 25 in MA)
    - :code:`horizon` (int = None): The maximum length of each episode. Set to None to remove constraint. (default to 1000 in MA, see :ref:`rl_environments:Multi-agent Environments`)



Reward, Cost and Termination Function Config
##############################################

There are a lot of coefficients to describe the reward function and cost function.
You can adjust the default reward function or design your own functions.
Please refer to :ref:`reward_cost_and_termination_function:Reward Function`, :ref:`reward_cost_and_termination_function:Cost Function` and :ref:`reward_cost_and_termination_function:Termination Function` for more information.


Engine Config
################

This is the engine core config of MetaDrive, including physics engine, window size and so on.
We don't suggest to modify this part if you are not confident on what you are doing.

    - :code:`window_size` (tuple): width and height of interface window. Default is (1200, 900).
    - :code:`physics_world_step_size` (float = 0.02): the minimum time interval between two time steps of bullet physics engine.
    - :code:`show_fps` (bool = True): Turn on/ turn off the frame rater.
    - :code:`debug_physics_world` (bool = False): if True then only render physics world without model
    - :code:`debug_static_world` (bool = True): if True then merge the static world and dynamic world to one world and render this world
    - :code:`pstats` (bool = False): if True then use Panda3D built-in debug tool to profile the program
    - :code:`global_light` (bool = False): True to enable global light. It will consume more computation resource to render.
    - :code:`debug` (bool = False): for developing use, draw the scene with bounding box


Default Config
################

The default config dicts are widely spread in many files. The basic config about some general setting is provided in the `BaseEnv Class <https://github.com/metadriverse/metadrive/blob/main/metadrive/envs/base_env.py>`_.
More detailed config is provided in the `MetaDriveEnv Class <https://github.com/metadriverse/metadrive/blob/main/metadrive/envs/metadrive_env.py>`_.
Besides, for `SafeMetaDriveEnv Class <https://github.com/metadriverse/metadrive/blob/main/metadrive/envs/safe_metadrive_env.py>`_
and `MultiAgentMetaDrive Class <https://github.com/metadriverse/metadrive/blob/main/metadrive/envs/marl_envs/multi_agent_metadrive.py>`_
there also have many task-specified config. Please feel free to open issues if you have any question about the environmental settings!



